# SpringPot Model: Power-Law Relaxation Fitting

This notebook demonstrates the complete workflow for fitting the SpringPot (fractional element) model to power-law relaxation data, showcasing fractional viscoelasticity with Bayesian uncertainty quantification.

## Learning Objectives

- Fit SpringPot model to power-law relaxation data
- Understand fractional derivative concepts (alpha parameter)
- Analyze parameter correlations (c_alpha vs alpha)
- Perform Bayesian inference with warm-start workflow
- Interpret ArviZ diagnostics for fractional models

**Prerequisites:** Basic rheology, complete 01-maxwell-fitting.ipynb first

**Estimated Time:** 35-45 minutes

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

from rheojax.pipeline.base import Pipeline
from rheojax.core.data import RheoData
from rheojax.models.springpot import SpringPot
from rheojax.core.jax_config import safe_import_jax, verify_float64

jax, jnp = safe_import_jax()
verify_float64()
print(f"✓ JAX float64 precision enabled")

np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## SpringPot Theory

The SpringPot (Scott-Blair element) interpolates between elastic solid (α=1) and viscous fluid (α=0):

**Relaxation Modulus:**
$$G(t) = \frac{c_\alpha \cdot t^{-\alpha}}{\Gamma(1-\alpha)}$$

**Parameters:**
- $c_\alpha$ = material constant (Pa·s$^\alpha$)
- $\alpha$ = power-law exponent (0=fluid, 1=solid)

**Physical Interpretation:**
- α=0: Pure viscous (Newtonian fluid)
- α=0.5: Balanced viscoelastic (typical gels)
- α=1: Pure elastic (Hookean solid)

**Applications:** Soft biological tissues, hydrogels, polymer melts with broad relaxation spectrum

## Generate Synthetic Data

In [ ]:
# True SpringPot parameters
c_alpha_true = 1e5  # Pa·s^alpha
alpha_true = 0.4    # Power-law exponent

print(f"True Parameters:")
print(f"  c_alpha = {c_alpha_true:.2e} Pa·s^{alpha_true}")
print(f"  alpha   = {alpha_true:.2f}")

# Time array (logarithmic)
t = np.logspace(-2, 2, 50)

# True relaxation modulus
from scipy.special import gamma as scipy_gamma
G_t_true = c_alpha_true * t**(-alpha_true) / scipy_gamma(1 - alpha_true)

# Add noise
noise_level = 0.015
noise = np.random.normal(0, noise_level * G_t_true)
G_t_noisy = G_t_true + noise

print(f"\nData: {len(t)} points, noise {noise_level*100:.1f}%")

In [ ]:
# Visualize power-law decay
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.loglog(t, G_t_noisy, 'o', alpha=0.7, label='Data')
ax1.loglog(t, G_t_true, '--', linewidth=2, alpha=0.4, label='True')
ax1.set_xlabel('Time (s)', fontweight='bold')
ax1.set_ylabel('G(t) (Pa)', fontweight='bold')
ax1.set_title('Power-Law Relaxation', fontweight='bold')
ax1.grid(True, alpha=0.3, which='both')
ax1.legend()

# Log-log slope = -alpha
log_t = np.log10(t)
log_G = np.log10(G_t_noisy)
slope, intercept = np.polyfit(log_t[10:40], log_G[10:40], 1)
ax2.plot(log_t, log_G, 'o', alpha=0.7, label='Data')
ax2.plot(log_t, slope*log_t + intercept, '--', linewidth=2, label=f'Fit: slope={slope:.3f}')
ax2.set_xlabel('log₁₀(t)', fontweight='bold')
ax2.set_ylabel('log₁₀(G)', fontweight='bold')
ax2.set_title(f'Power-Law: slope ≈ -α = {slope:.3f}', fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

print(f"\nEstimated alpha from slope: {-slope:.3f} (true: {alpha_true:.3f})")

## Pipeline API Fitting

In [ ]:
data = RheoData(x=t, y=G_t_noisy, x_units='s', y_units='Pa', domain='time')

pipeline = Pipeline(data)
pipeline.fit('springpot')

params = pipeline.get_fitted_parameters()
c_alpha_fit = params['c_alpha']
alpha_fit = params['alpha']

print("="*60)
print("PIPELINE API RESULTS")
print("="*60)
print(f"c_alpha = {c_alpha_fit:.4e} (true: {c_alpha_true:.4e})")
print(f"alpha   = {alpha_fit:.4f} (true: {alpha_true:.4f})")
print(f"\nErrors: c_alpha {abs(c_alpha_fit-c_alpha_true)/c_alpha_true*100:.2f}%, alpha {abs(alpha_fit-alpha_true)/alpha_true*100:.2f}%")
print("="*60)

## Modular API Fitting

In [ ]:
model = SpringPot()
model.parameters.set_bounds('c_alpha', (1e3, 1e7))
model.parameters.set_bounds('alpha', (0.1, 0.9))

model.fit(t, G_t_noisy)

c_alpha_mod = model.parameters.get_value('c_alpha')
alpha_mod = model.parameters.get_value('alpha')

print("="*60)
print("MODULAR API RESULTS")
print("="*60)
print(f"c_alpha = {c_alpha_mod:.4e} (true: {c_alpha_true:.4e})")
print(f"alpha   = {alpha_mod:.4f} (true: {alpha_true:.4f})")
print("="*60)

assert np.allclose(c_alpha_fit, c_alpha_mod, rtol=1e-6)
assert np.allclose(alpha_fit, alpha_mod, rtol=1e-6)
print("\n✓ Pipeline and Modular APIs identical")

## Results Visualization

In [ ]:
G_t_pred = model.predict(t)
residuals = G_t_noisy - G_t_pred
rel_residuals = residuals / G_t_noisy * 100

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.loglog(t, G_t_noisy, 'o', alpha=0.7, label='Data')
ax1.loglog(t, G_t_true, '--', linewidth=2, alpha=0.4, label='True')
ax1.loglog(t, G_t_pred, '-', linewidth=2.5, label='Fitted')
ax1.set_xlabel('Time (s)', fontweight='bold')
ax1.set_ylabel('G(t) (Pa)', fontweight='bold')
ax1.set_title('SpringPot Fit', fontweight='bold')
ax1.grid(True, alpha=0.3, which='both')
ax1.legend()

ax2.semilogx(t, rel_residuals, 'o', alpha=0.7)
ax2.axhline(0, color='black', linestyle='--', alpha=0.5)
ax2.axhline(noise_level*100, color='red', linestyle=':', linewidth=1.5, alpha=0.5, label=f'±{noise_level*100:.1f}%')
ax2.axhline(-noise_level*100, color='red', linestyle=':', linewidth=1.5, alpha=0.5)
ax2.set_xlabel('Time (s)', fontweight='bold')
ax2.set_ylabel('Relative Residuals (%)', fontweight='bold')
ax2.set_title('Residual Analysis', fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

r2 = 1 - np.sum(residuals**2) / np.sum((G_t_noisy - np.mean(G_t_noisy))**2)
print(f"\nR² = {r2:.6f}")
print(f"Mean |residual| = {np.mean(np.abs(rel_residuals)):.2f}%")

## Bayesian Inference

In [ ]:
print("="*60)
print("BAYESIAN INFERENCE")
print("="*60)
print("Running MCMC... (1-2 min)\n")

result = model.fit_bayesian(
    t, G_t_noisy,
    num_warmup=1000,
    num_samples=2000,
    num_chains=1,
    initial_values={
        'c_alpha': model.parameters.get_value('c_alpha'),
        'alpha': model.parameters.get_value('alpha')
    }
)

print("\nComplete!")
print("="*60)

In [ ]:
posterior = result.posterior_samples
diagnostics = result.diagnostics
summary = result.summary

credible = model.get_credible_intervals(posterior, credibility=0.95)

print("="*60)
print("POSTERIOR SUMMARY")
print("="*60)
print(f"\nc_alpha = {summary['c_alpha']['mean']:.4e} ± {summary['c_alpha']['std']:.4e}")
print(f"alpha   = {summary['alpha']['mean']:.4f} ± {summary['alpha']['std']:.4f}")
print(f"\n95% Credible Intervals:")
print(f"  c_alpha: [{credible['c_alpha'][0]:.4e}, {credible['c_alpha'][1]:.4e}]")
print(f"  alpha:   [{credible['alpha'][0]:.4f}, {credible['alpha'][1]:.4f}]")
print(f"\nConvergence:")
print(f"  R-hat (c_alpha): {diagnostics['r_hat']['c_alpha']:.4f} {'✓' if diagnostics['r_hat']['c_alpha']<1.01 else '✗'}")
print(f"  R-hat (alpha):   {diagnostics['r_hat']['alpha']:.4f} {'✓' if diagnostics['r_hat']['alpha']<1.01 else '✗'}")
print(f"  ESS (c_alpha):   {diagnostics['ess']['c_alpha']:.0f} {'✓' if diagnostics['ess']['c_alpha']>400 else '✗'}")
print(f"  ESS (alpha):     {diagnostics['ess']['alpha']:.0f} {'✓' if diagnostics['ess']['alpha']>400 else '✗'}")
print("="*60)

## ArviZ Diagnostics (Focused)

For SpringPot, parameter correlation (c_alpha vs alpha) is critical to check.

In [ ]:
import arviz as az

idata = result.to_inference_data()

# Trace plot
az.plot_trace(idata, figsize=(12, 6))
plt.tight_layout()
plt.show()

In [ ]:
# Pair plot - critical for fractional models
az.plot_pair(
    idata,
    var_names=['c_alpha', 'alpha'],
    kind='scatter',
    divergences=True,
    figsize=(10, 8)
)
plt.tight_layout()
plt.show()

print("""
CRITICAL: c_alpha and alpha often show strong negative correlation!
- Both parameters affect power-law slope
- Correlation indicates partial non-identifiability
- Multi-technique fitting can improve constraint
""")

## Physical Interpretation

**Alpha Parameter:**
- α ≈ 0.4: Predominantly viscous (gel-like)
- Physical meaning: Degree of solid-like vs fluid-like character

**Material Constant c_α:**
- Sets magnitude of stress response
- Units depend on α: Pa·s$^\alpha$

**Applications:**
- Biological tissues (0.1 < α < 0.5)
- Polymer melts with broad relaxation spectrum
- Soft gels and hydrogels

**Comparison to Classical Models:**
- Maxwell (α→0): Complete stress relaxation
- Elastic solid (α=1): No relaxation
- SpringPot: Power-law interpolation

## Key Takeaways

1. **Fractional Viscoelasticity:**
   - α parameter controls solid/fluid character
   - Power-law decay ⇒ broad relaxation spectrum
   - Often more realistic than single exponential

2. **Parameter Correlation:**
   - c_alpha and alpha typically correlated
   - Check pair plot for non-identifiability
   - Multi-technique data improves constraint

3. **Bayesian Benefits:**
   - Quantifies correlation uncertainty
   - Reveals parameter non-identifiability
   - Essential for fractional models

## Next Steps

- **[advanced/04-fractional-models-deep-dive.ipynb](../advanced/04-fractional-models-deep-dive.ipynb)**: All 11 fractional models
- **[bayesian/04-model-comparison.ipynb](../bayesian/04-model-comparison.ipynb)**: Compare SpringPot vs Maxwell
- **[advanced/01-multi-technique-fitting.ipynb](../advanced/01-multi-technique-fitting.ipynb)**: Reduce parameter correlation

In [ ]:
# Session info
import sys
import rheojax
print(f"Python: {sys.version}")
print(f"Rheo: {rheojax.__version__}")
print(f"JAX: {jax.__version__}")